In [1]:
from astroquery.eso import Eso
from astropy.coordinates import SkyCoord
import astropy.units as u 

from astroquery.alma import Alma as ALMA
from astroquery.cadc import Cadc as CADC
from astroquery.esa.jwst import Jwst as JWST
from astroquery.esa.hubble import ESAHubble as HST
from astroquery.mast import Observations as MAST

In [2]:
eso = Eso() # create an instance of the ESO class 
alma = ALMA()
hst = HST()
jwst = JWST()
cadc = CADC()
mast = MAST()

In [3]:
alma.query_region?

Signature: alma.query_region(self, *args, **kwargs)
Docstring:
Queries the service and returns a table object.

Query the ALMA archive with a source name and radius

Parameters
----------
coordinates : str / `astropy.coordinates`
    the identifier or coordinates around which to query.
radius : str / `~astropy.units.Quantity`, optional
    the radius of the region
public : bool
    True to return only public datasets, False to return private only,
    None to return both
science : bool
    True to return only science datasets, False to return only
    calibration, None to return both
payload : dict
    Dictionary of additional keywords.  See `help`.
enhanced_results : bool
    True to return a table with quantities instead of just values. It
    also returns the footprints as `regions` objects.

Returns
-------
table : A `~astropy.table.Table` object.
File:      /opt/anaconda3/envs/tst/lib/python3.9/site-packages/astroquery/utils/process_asyncs.py
Type:      function

In [4]:
jwst.query_region?

Signature:
jwst.query_region(
    coordinate,
    *,
    radius=None,
    width=None,
    height=None,
    observation_id=None,
    cal_level='Top',
    prod_type=None,
    instrument_name=None,
    filter_name=None,
    proposal_id=None,
    only_public=False,
    show_all_columns=False,
    async_job=False,
    verbose=False,
)
Docstring:
Launches a query region job in sync/async mode
TAP & TAP+

Parameters
----------
coordinate : astropy.coordinate, mandatory
    coordinates center point
radius : astropy.units, required if no 'width' nor 'height'
    are provided
    radius (deg)
width : astropy.units, required if no 'radius' is provided
    box width
height : astropy.units, required if no 'radius' is provided
    box height
observation_id : str, optional, default None
    get the observation given by its ID.
cal_level : object, optional, default 'Top'
    get the planes with the given calibration level. Options are:
    'Top': str, only the planes with the highest calibration level

In [5]:
mast.query_region?

Signature: mast.query_region(self, *args, **kwargs)
Docstring:
Queries the service and returns a table object.

Given a sky position and radius, returns a list of MAST observations.
See column documentation `here <https://mast.stsci.edu/api/v0/_c_a_o_mfields.html>`__.

Parameters
----------
coordinates : str or `~astropy.coordinates` object
    The target around which to search. It may be specified as a
    string or as the appropriate `~astropy.coordinates` object.
radius : str or `~astropy.units.Quantity` object, optional
    Default 0.2 degrees.
    The string must be parsable by `~astropy.coordinates.Angle`. The
    appropriate `~astropy.units.Quantity` object from
    `~astropy.units` may also be used. Defaults to 0.2 deg.
pagesize : int, optional
    Default None.
    Can be used to override the default pagesize for (set in configs) this query only.
    E.g. when using a slow internet connection.
page : int, optional
    Default None.
    Can be used to override the default behav

In [6]:
cadc.query_region?

Signature: cadc.query_region(self, *args, **kwargs)
Docstring:
Queries the service and returns a table object.

Queries the CADC for a region around the specified coordinates.

Parameters
----------
coordinates : str or `astropy.coordinates`.
    coordinates around which to query
radius : str or `astropy.units.Quantity`.
    the radius of the cone search
collection: Name of the CADC collection to query, optional
get_query_payload : bool, optional
    Just return the dict of HTTP request parameters.

Returns
-------
table : A `~astropy.table.Table` object.
File:      /opt/anaconda3/envs/tst/lib/python3.9/site-packages/astroquery/utils/process_asyncs.py
Type:      function

In [7]:
def query_region(coordinates, radius=1.0 * u.arcmin, instrument='', product_type='science'):
    """
    Queries the ESO archive for observations within a given sky region.

    Parameters
    ----------
    coordinates : str or astropy.coordinates.SkyCoord
        The target name (e.g., 'Sgr A*') or coordinates (RA, Dec) around which to query.
    radius : astropy.units.Quantity, optional
        The search radius around the given coordinates. Default is 1 arcminute.
    instrument : str, optional
        The instrument name to filter results (e.g., 'HAWKI'). Default is an empty string (no filter).
    product_type : str, optional
        The type of data product to query. Options:
        - 'science' (default): Science-ready processed data.
        - 'raw': Raw observational data.
        - 'raw_instrument': Raw data filtered by the instrument-specific table.

    Returns
    -------
    astropy.table.Table
        A table of matching ESO archive entries.
    
    Raises
    ------
    ValueError
        If an invalid product_type is provided.
    """

    # Validate product_type
    valid_types = ['science', 'raw', 'raw_instrument']
    if product_type not in valid_types:
        raise ValueError(f'Invalid product type. Choose from {valid_types}.')

    # Convert coordinates from string to SkyCoord if needed
    if isinstance(coordinates, str):
        coordinates = SkyCoord.from_name(coordinates)

    # Convert coordinates to RA, Dec if needed
    coordinates = coordinates.fk5 
    # coordinates = coordinates.icrs

    # Ensure radius is properly handled
    if not isinstance(radius, u.Quantity):
        raise TypeError("Radius must be an astropy Quantity with angular units (e.g., '1.0 * u.arcmin').")

    # Query based on product type
    if product_type == 'science':
        table = eso.query_collections(
            instrument,
            ra=coordinates.ra.deg,
            dec=coordinates.dec.deg,
            radius=radius.to(u.deg).value
        )
    
    elif product_type == 'raw':
        table = eso.query_main(
            column_filters={'instrument': instrument},
            ra=coordinates.ra.deg,
            dec=coordinates.dec.deg,
            radius=radius.to(u.deg).value
        )

    elif product_type == 'raw_instrument':
        table = eso.query_instrument(
            instrument,
            ra=coordinates.ra.deg,
            dec=coordinates.dec.deg,
            radius=radius.to(u.deg).value
        )

    return table

In [8]:
coordinates_name = 'Sgr A*' # name of the source
coordinates = SkyCoord.from_name(coordinates_name) # create a SkyCoord object from the name of the source
coordinates_galactic = coordinates.galactic # convert the coordinates to galactic coordinates

radius = 20 *u.arcsec # radius of the search region
instrument = 'HAWKI'

In [9]:
# Example Usages 
result_table = query_region(coordinates_name, radius=radius, instrument=instrument)
result_table = query_region(coordinates, radius=radius, instrument=instrument)
result_table = query_region(coordinates_galactic, radius=radius, instrument=instrument)
result_table = query_region(coordinates, radius=radius, instrument=instrument, product_type='raw')
result_table = query_region(coordinates, radius=radius, instrument=instrument, product_type='raw_instrument')